## Import  modules and datasets 

In [1]:
import pandas as pd
import random
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

##### Initialize the spotify user account

In [2]:
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(
    client_id="320d32b1cdff443d9f8498b16f9cf9ea",
    client_secret="290b630ee62d4ce19b3b244984190fa6"))

Note: These lists haven been  created in this project using web scrapping and spotify API

In [4]:
hits_now = pd.read_csv("./Data/hits.csv",index_col=0)
hits_old = pd.read_csv("./Data/old_hits.csv",index_col=0)
old_artists = pd.read_csv("./Data/old_artists.csv",index_col=0)
songs_1000 = pd.read_csv("./Data/songs_1000_clusters.csv", index_col=0)
everybody_df = pd.read_csv("./Data/everybody_df.csv", index_col=0)
hits_old.columns = ["70s","80s","90s","00s","10s"]
old_artists.columns = ["70s","80s","90s","00s","10s"]

-------------------------

## Prepare the clustering

Note: The clusters are already created and added to the songs_1000 dataframe, but we need to fit the scaler with the data from these songs in order to fit also the kmeans function. Doing this, we will be able to assign the input song to one of the already created clusters.

In [5]:
songs_1000_toclusters = songs_1000.iloc[:,2:7]
songs_1000_toclusters.head()

,Energy,Tempo,Danceability,Instrumentalness,Valence
0,0.257,75.752,0.547,0.183000,0.169
1,0.660,149.813,0.249,0.002600,0.435
2,0.685,120.965,0.537,0.000000,0.542
3,0.487,117.072,0.521,0.000000,0.483
4,0.206,79.764,0.149,0.000649,0.264


In [9]:
scaler = StandardScaler()
scaler.fit(songs_1000_toclusters)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [10]:
tracks_scaled = scaler.transform(songs_1000_toclusters)
tracks_scaled_df = pd.DataFrame(tracks_scaled)

In [12]:
kmeans = KMeans()
kmeans.fit(tracks_scaled)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=8, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

--------

## Design the whole flow

##### 1. Ask for an input, if it is an actual hit, return another one. If it is an old hit, return one from the same decade.

In [88]:
print("Hello! Please, write a song or artist you like. \n")
request = input().title()

if request in list(hits_now["song"]):
    title=random.choice(hits_now["song"])
    artist = hits_now[hits_now["song"]==title]["artist"].values[0]
    print("Cool! Maybe you would like this song: "+'\033[1m'+ title +'\033[0m'+ " by " +'\033[1m'+ artist)
    
    
elif request in list(hits_old["10s"]):
    title=random.choice(hits_old["10s"])
    artist = old_artists[hits_old["10s"]==title]["10s"].values[0]
    print("Great song from last decade! \nThis one was also great: "+'\033[1m'+ title +'\033[0m'+ " by " +'\033[1m'+ artist)
    
    
elif request in list(hits_old["00s"]):
    title=random.choice(hits_old["00s"])
    artist = old_artists[hits_old["00s"]==title]["00s"].values[0]
    print("Great song from the 2000s! \nThis one was also great: "+'\033[1m'+ title +'\033[0m'+ " by " +'\033[1m'+ artist)
    
        
        
elif request in list(hits_old["90s"]):
    title=random.choice(hits_old["90s"])
    artist = old_artists[hits_old["90s"]==title]["90s"].values[0]
    print("Great song from the 90s! \nThis one was also great: "+'\033[1m'+ title +'\033[0m'+ " by " +'\033[1m'+ artist)
    
        
        
elif request in list(hits_old["80s"]):
    title=random.choice(hits_old["80s"])
    artist = old_artists[hits_old["80s"]==title]["80s"].values[0]
    print("Great song from the 80s! \nThis one was also great: "+'\033[1m'+ title +'\033[0m'+ " by " +'\033[1m'+ artist)
    
        
        
elif request in list(hits_old["70s"]):
    title=random.choice(hits_old["70s"])
    artist = old_artists[hits_old["70s"]==title]["70s"].values[0]
    print("Great song from the 70s! \nThis one was also great: "+'\033[1m'+ title +'\033[0m'+ " by " +'\033[1m'+ artist)
    
    
    
else:
    print("hey")#substitute here for the clustering-based selection 

Hello! Please, write a song or artist you like. 

vogue
Great song from last decade! 
This one was also great: Promiscuous by Nelly Furtado Featuring Timbaland


##### 2. If the input song is not an actual hit either an old hit, cluster the song and return another one from the same cluster from our database.

In [47]:
def get_recomendation_cluster(request):
    
    song = spotify.search(q=request, limit=1)
    song_URI = song["tracks"]["items"][0]["uri"]
    
    features = spotify.audio_features(song_URI)[0]
    feat = pd.DataFrame({"energy":features["energy"],
           "tempo":features["tempo"],
           "danceabilty":features["danceability"],
           "instrumentalness":features["instrumentalness"],
           "valence":features["valence"]},index=[0])
    
    feat_scaled = pd.DataFrame(scaler.transform(feat))
    feat_cluster = kmeans.predict(feat_scaled)
    cluster = feat_cluster[0]
    
    one_cluster = songs_1000[songs_1000["Cluster"]==cluster]
    title = one_cluster.sample()["Title"].values[0]
    artist = songs_1000[songs_1000["Title"]==title]["Artist"].values[0]
    print("Great! Your should listen this: "+'\033[1m' + title +'\033[0m'+ " by "+'\033[1m'+ artist)

In [48]:
get_recomendation_cluster("rabiosa")

Great! Your should listen this: Sexual Healing by Marvin Gaye


##### 3. If the input is a symbol (eg. *, +, etc) or an unidentified song, return one song from the "songs known by everybody" list.

In [36]:
try:
    get_recomendation_cluster(request)
    
except:
    title = everybody_df.sample()["Title"].values[0]
    artist = everybody_df[everybody_df["Title"]==title]["Artist"].values[0]
    print("We couldn't recognize your song, but we are sure you will love this: "+'\033[1m' + title + " by "+ artist)

Great! Your should listen this: War Pigs by Black Sabbath


-------

## Create a function with everythin

In [142]:
def song_recomendator():
    print("Hello! Please, write a song you like. \n")
    request = input().title()

    if request in list(hits_now["song"]):
        title=random.choice(hits_now["song"])
        artist = hits_now[hits_now["song"]==title]["artist"].values[0]
        print("Cool! Maybe you would like this song: "+'\033[1m'+ title +'\033[0m'+ " by " +'\033[1m'+ artist)


    elif request in list(hits_old["10s"]):
        title=random.choice(hits_old["10s"])
        artist = old_artists[hits_old["10s"]==title]["10s"].values[0]
        print("Great song from last decade! \nThis one was also great: "+'\033[1m'+ title +'\033[0m'+ " by " +'\033[1m'+ artist)


    elif request in list(hits_old["00s"]):
        title=random.choice(hits_old["00s"])
        artist = old_artists[hits_old["00s"]==title]["00s"].values[0]
        print("Great song from the 2000s! \nThis one was also great: "+'\033[1m'+ title +'\033[0m'+ " by " +'\033[1m'+ artist)



    elif request in list(hits_old["90s"]):
        title=random.choice(hits_old["90s"])
        artist = old_artists[hits_old["90s"]==title]["90s"].values[0]
        print("Great song from the 90s! \nThis one was also great: "+'\033[1m'+ title +'\033[0m'+ " by " +'\033[1m'+ artist)



    elif request in list(hits_old["80s"]):
        title=random.choice(hits_old["80s"])
        artist = old_artists[hits_old["80s"]==title]["80s"].values[0]
        print("Great song from the 80s! \nThis one was also great: "+'\033[1m'+ title +'\033[0m'+ " by " +'\033[1m'+ artist)



    elif request in list(hits_old["70s"]):
        title=random.choice(hits_old["70s"])
        artist = old_artists[hits_old["70s"]==title]["70s"].values[0]
        print("Great song from the 80s! \nThis one was also great: "+'\033[1m'+ title +'\033[0m'+ " by " +'\033[1m'+ artist)
    

    else:
        try:
            get_recomendation_cluster(request)
    
        except:
            title = everybody_df.sample()["Title"].values[0]
            artist = everybody_df[everybody_df["Title"]==title]["Artist"].values[0]
            print("We couldn't recognize your song, but we are sure you will love this one: " + '\033[1m'+ title +'\033[0m'+ " by "+'\033[1m'+ artist)

In [188]:
song_recomendator()

Hello! Please, write a song you like. 

vogue
Great song from the 90s! 
This one was also great: Far Away by Nickelback
